# _Classical_ Evaluations with Scorebook

Scorebook, developed by Trismik, is an open-source Python library for model evaluation. It supports both Trismik’s adaptive testing and traditional classical evaluations. In a classical evaluation, a model runs inference on every item in a dataset, and the results are scored using Scorebook’s built-in metrics, such as accuracy, to produce evaluation results.

Custom metrics can be implemented and integrated to suit specific evaluation needs.

Scorebook also enables evaluation across a grid or list of hyperparameter configurations, streamlining model optimization.

Evaluation results can be automatically uploaded to the Scorebook dashboard, organized by project, for storing, managing, and visualizing model evaluation experiments.

## Setup Trismik Credentials

The results from a classical evaluation can optionally be uploaded to a Trismik project on the dashboard, for the storing, managing, and visualization of evaluation results.

If logged in, the `evaluate` function will require a trismik project and experiment id.

Enter your trismik API key below.

In [ ]:
from scorebook import create_project, login

# Set your API key here and run this cell to login
TRISMIK_API_KEY = "your-trismik-api-key-here"

login(TRISMIK_API_KEY)
print("✓ Logged in to Trismik")

# Create a project
project = create_project(
    name = "Classical Evaluation Demo",
    description = "A project created as part of Trismik's quick-start guides."
)

print("✓ Project created")
print(f"Project ID: {project.id}")



## Evaluation Datasets

A scorebook evaluation requires an evaluation dataset, represented by the `EvalDataset` class. Evaluation datasets can be constructed via a number of factory methods. In this example we will create a basic evaluation dataset from a list of evaluation items.

In [ ]:
from scorebook import EvalDataset
from scorebook.metrics.accuracy import Accuracy

# Create a sample dataset from a list of multiple-choice questions
evaluation_items = [
    {"question": "What is 2 + 2?", "answer": "4"},
    {"question": "What is the capital of France?", "answer": "Paris"},
    {"question": "Who wrote Romeo and Juliet?", "answer": "William Shakespeare"},
    {"question": "What is the chemical symbol for gold?", "answer": "Au"}
]

# Create an EvalDataset from the list
dataset = EvalDataset.from_list(
    name = "sample_multiple_choice",
    metrics = Accuracy,
    items = evaluation_items,
    input = "question",
    label = "answer",
)

print(f"✓ Created dataset with {len(dataset.items)} items")

## Preparing Models for Evaluation

To evaluate a model with Scorebook, it must be encapsulated within an inference function. An inference function must accept a list of model inputs, pass these to the model for inference, collect and return outputs generated.

### Instantiate a Local Qwen Model

For this quick-start guide, we will use the lightweight Qwen2.5 0.5B instruct model, via Hugging Face's transformers package.

In [ ]:
import transformers

# Instantiate a model
pipeline = transformers.pipeline(
    "text-generation",
    model="Qwen/Qwen2.5-0.5B-Instruct",
    model_kwargs={"torch_dtype": "auto"},
    device_map="auto",
)

print("✓ Transformers pipeline instantiated")

### Define an Inference Function

An inference function can be defined to encapsulate any model, local or cloud-hosted. There is flexibility in how an inference function can be defined, the only requirements are the function signature. An inference function must,

Accept:

- A list of model inputs.
- Hyperparameters which can be optionally accessed via kwargs.

Return

- A list of parsed model outputs for scoring.


In [ ]:
# Define an inference function
from typing import Any, List

def qwen(inputs: List[Any], **hyperparameters: Any) -> List[Any]:
    """Run inference on a list of inputs using the 0.5B Qwen model."""
    inference_outputs = []

    for model_input in inputs:
        messages = [
            {"role": "system", "content": hyperparameters.get("system_message", "You are a helpful assistant.")},
            {"role": "user", "content": str(model_input)},
        ]

        output = pipeline(
            messages,
            temperature = hyperparameters.get("temperature", 0.7),
            top_p = hyperparameters.get("top_p", 0.9),
            top_k = hyperparameters.get("top_k", 50),
            max_new_tokens = 512,
            do_sample = hyperparameters.get("temperature", 0.7) > 0,
        )

        inference_outputs.append(output[0]["generated_text"][-1]["content"])

    return inference_outputs

print("✓ Inference function for Qwen2.5 0.5B defined")
print(qwen(["Hello!"]))

## Hyperparameters

Scorebook makes it easy to evaluate a model across multiple hyperparameter configurations. Hyperparameters evaluated can be specified as

1. **A Single Hyperparameter Configuration.**
```python
hyperparameters = {'temperature': 0.9, 'top_p': 0.8, 'top_k': 40}
```

2. **A List of Hyperparameter Configurations.**
```python
hyperparameters = [
    {'temperature': 0.9, 'top_p': 0.8, 'top_k': 40},
    {'temperature': 0.7, 'top_p': 0.6, 'top_k': 20},
]
```

3. **A Grid of Hyperparameters**

```python
hyperparameters = {
    "temperature": [0.3, 0.9],
    "top_p": [0.6, 0.9],
    "top_k": [20, 40],
}
```

## Running an Evaluation

Running a scorebook evaluation with `evaluate` only requires an inference function and a dataset.

In [ ]:
from scorebook import evaluate

# Run evaluation
results = evaluate(
    inference= qwen,
    datasets = dataset,
    hyperparameters = {
        'temperature': 0.9,
        'top_p': 0.8,
        'top_k': 40,
        'system_message': ["Answer the question directly, provide no additional context."]
    },
    experiment_id = "Classical-Evaluation-Demo",
    project_id = project.id,
)

print("Qwen2.5 0.5B Evaluation Results:")
print(f"accuracy: {results[0]['accuracy']}")

## Next Steps

- [Scorebook docs](https://docs.trismik.com/scorebook/introduction-to-scorebook/): Scorebook's full documentation.
- [Scorebook repository](https://github.com/trismik/scorebook): Scorebook is an open-source library, view the code and more examples.